## Multi-Agent Systems with Strands Agents leveraging MCP tools



While building agents we have an option to use the custom tools and the predefined tools provided by the Strands SDK like **retrieve**, **calculator**, etc. However it might be more beneficial from the engineering and scale perspective to augment those tools with the tools hosted via MCP server, and reuse those tools across multiple agents

In this section, we will explore the different types of multi-agent systems you can build with **Strands Agents** and also leveraging MCP tools to augment the functionality of the application.

The architecture will loook like below

![Hierarchical Mortgage Agent with MCP tools integration](../../images/strands-multi-agent-mcp.png)

## multi-agent collaboration

Multi-agent collaboration enables multiple Amazon Bedrock Agents to collaboratively plan and solve complex tasks. With multi-agent collaboration, you can quickly assemble a team of agents that can break down tasks, assign specific tasks to domain specialist sub-agents, work in parallel, and leverage each other's strengths, which leads to more efficient problem-solving. Multi-agent provides a centralized mechanism for planning, orchestration , and user interaction for your generative AI applications.

You can describe each agent’s role and responsibilities using natural language.

In this lab we will be creating 2 additional agents. Similar to how we created a stand-alone agent in the previous lab. 

2. **Collaborator agent 1** – Responsible for handling general mortgages.

3. **Collaborator agent 2** – Responsible for handling existing questions.

In [ ]:
!pip install strands-agents strands-agents-tools mcp

In [ ]:
import os
import time
import boto3
import logging
import botocore
import json
from textwrap import dedent
import sys
import pickle
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client



from strands import Agent, tool
from strands_tools import retrieve, calculator

#from utils.kb_helper import KnowledgeBaseHelper


sys.path.insert(0, '..') 

%load_ext autoreload
%autoreload 2

from src.utils.secrets_helper import SecretsHelper
secrets_helper = SecretsHelper(region_name="us-east-1")

In [2]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)
boto3.set_stream_logger(name='botocore.credentials', level=logging.DEBUG)

In [3]:
agent_foundation_models = [ 
    "amazon.nova-lite-v1:0",
    "us.anthropic.claude-3-5-sonnet-20241022-v2:0"
    ]

In [4]:
%store -r kb_id
print("KnowledgeBase ID:",kb_id)

KnowledgeBase ID: OJOFGNEAFI


In [5]:
# Initialize knowledge base helper
# Set knowledge base ID as environment variable so that retrieve tool can use it
os.environ["KNOWLEDGE_BASE_ID"] = kb_id

#use the kb_id to initialise the KB helper object
#kb_helper = KnowledgeBaseHelper(kb_id)

In [6]:
value = os.getenv('KNOWLEDGE_BASE_ID')
print(value)

OJOFGNEAFI


In [22]:
!echo $KNOWLEDGE_BASE_ID

OJOFGNEAFI


# 1. Create the agent for general mortgage questions.

We will use the helper Agent class which provides methods for creating, configuring, and invoking individual Agents, including
associating them with Guardrails, Knowledge Bases, and Tools.

For this Agent we will also associate our Knowledge base created earlier. 

In [23]:
@tool
def answer_general_mortgage_questions(query):
    # Create the General Mortgage Agent
    general_mortgage_agent = Agent(
        model="anthropic.claude-3-sonnet-20240229-v1:0",
        tools=[
           # Pass the knowledge base ID to the retrieve tool
           lambda query, **kwargs: retrieve(query, knowledge_base_id=kb_id, **kwargs)
        ],
        system_prompt="""
        You are a mortgage bot, and can answer questions about mortgage refinancing and tradeoffs of mortgage types. Greet the customer first. Respond to the greeting by another greeting    
        
        You can:
        1. Provide general information about mortgages
        2. Handle conversations about general mortgage questions, like high level concepts of refinincing or tradeoffs of 15-year vs 30-year terms.
        3. Offer guidance on the mortgage refinancing and tradeoffs of mortgage types.
        4. Access a knowledge base of mortgage information
        5. Only answer from the knowledge base and not from your general knowledge. If you dont have the answer from Knowledge base, say "I dont know"
        
        When helping users:
        - Provide clear explanations
        - Use plain language to explain complex financial terms
        - Offer balanced advice considering both pros and cons
        - Be informative without making specific financial recommendations
        - Use the knowledge base for detailed information to answer general questions about mortgages, like how to refinance, or the difference between 15-year and 30-year mortgages
        
        Remember that you're providing general mortgage information, not financial advice.
        Always clarify that users should consult with a financial advisor for personalized advice.
        """
    )
    return str(general_mortgage_agent(query))



In [24]:
#alternative approach to pass kb id to the retrieve tool
'''
@tool
def answer_general_mortgage_questions(query):
    # Create the General Mortgage Agent
    general_mortgage_agent = Agent(
        model="anthropic.claude-3-sonnet-20240229-v1:0",
        tools=[retrieve],
        system_prompt="""
        You are a mortgage bot, and can answer questions about mortgage refinancing and tradeoffs of mortgage types. Greet the customer first. Respond to the greeting by another greeting    
        
        You can:
        1. Provide general information about mortgages
        2. Handle conversations about general mortgage questions, like high level concepts of refinincing or tradeoffs of 15-year vs 30-year terms.
        3. Offer guidance on the mortgage refinancing and tradeoffs of mortgage types.
        4. Access a knowledge base of mortgage information
        5. Only answer from the knowledge base and not from your general knowledge. If you dont have the answer from Knowledge base, say "I dont know"
        
        When helping users:
        - Provide clear explanations
        - Use plain language to explain complex financial terms
        - Offer balanced advice considering both pros and cons
        - Be informative without making specific financial recommendations
        - Use the knowledge base for detailed information to answer general questions about mortgages, like how to refinance, or the difference between 15-year and 30-year mortgages
        
        Remember that you're providing general mortgage information, not financial advice.
        Always clarify that users should consult with a financial advisor for personalized advice.
        """
    )
    return str(general_mortgage_agent(query, tool_params={
                     "retrieve": {
                         "knowledge_base_id": kb_id,
                         "max_results": 5
                     }
                 }))

'''

'\n@tool\ndef answer_general_mortgage_questions(query):\n    # Create the General Mortgage Agent\n    general_mortgage_agent = Agent(\n        model="anthropic.claude-3-sonnet-20240229-v1:0",\n        tools=[retrieve],\n        system_prompt="""\n        You are a mortgage bot, and can answer questions about mortgage refinancing and tradeoffs of mortgage types. Greet the customer first. Respond to the greeting by another greeting    \n        \n        You can:\n        1. Provide general information about mortgages\n        2. Handle conversations about general mortgage questions, like high level concepts of refinincing or tradeoffs of 15-year vs 30-year terms.\n        3. Offer guidance on the mortgage refinancing and tradeoffs of mortgage types.\n        4. Access a knowledge base of mortgage information\n        5. Only answer from the knowledge base and not from your general knowledge. If you dont have the answer from Knowledge base, say "I dont know"\n        \n        When helpi

In [25]:
# Test the General agent and confirm that it consults the KB to answer the questions

In [ ]:
print(answer_general_mortgage_questions("What is the benefit of refinancing, if any?"))

## 2. Create the agent for existing mortgage questions.

Create the Agent for managing existing mortgages

In this section we will go through all the steps to create an Agent for Amazon Bedrock. 

These are the steps to complete:
    
1. Create the new agent (with the helper function taking care of IAM role creation)
2. Add an action group backed by a new Lambda function (with the helper function handling IAM role creation, Lambda function creation, adding the action group to the agent, and preparing the agent)

#### Create the Lambda function code
Here we create a source code file for a new Lambda function to implement the action group for our Existing Mortgage agent. Notice the **TODO** section in the code below. In this example we have hardcoded the response back from the agent but in your environment this is where your business logic would reside. 

In [27]:
@tool
def get_mortgage_details(customer_id):
    # TODO: Implement real business logic to retrieve mortgage status
    return {
        "account_number": customer_id,
        "outstanding_principal": 150000.0,
        "interest_rate": 4.5,
        "maturity_date": "2030-06-30",
        "payments_remaining": 72,
        "last_payment_date": "2024-06-01",
        "next_payment_due": "2024-07-01",
        "next_payment_amount": 1250.0
    }


In [28]:
@tool
def answer_existing_mortgage_questions(query):
    # Create the Existing Mortgage Agent
    existing_mortgage_agent = Agent(
        model="anthropic.claude-3-sonnet-20240229-v1:0",
        tools=[
           get_mortgage_details
        ],
        system_prompt="""
        You are an Existing Mortgage Assistant that helps customers with their current mortgages.

        You can:
        1. Provide information about a customer's existing mortgage
        2. Check mortgage status including balance and payment information
        3. Evaluate refinancing eligibility
        4. Calculate payoff timelines with extra payments
        5. Answer questions about mortgage terms and conditions

        When helping users:
        - Always verify the customer ID before providing information
        - Provide clear explanations of mortgage details
        - Format financial data in a readable way
        - Explain payment schedules and upcoming due dates
        - Offer guidance on refinancing options when appropriate
        - Use the knowledge base for detailed information when needed

        Remember that you're dealing with sensitive financial information, so maintain a professional tone
        and ensure accuracy in all responses.
        """
    )
    return str(existing_mortgage_agent(query))



Now we create the agent itself, giving it a name, 
a brief description, and most importantly, a set of instructions.

also briefly explain action group

we add an action group to the new agent

In [ ]:
#test

print(answer_existing_mortgage_questions("I'm customer 98991. when's my next payment due?"))

** Install the below MCP python package before running the MCP server**

In [ ]:
!pip install mcp

**Run the MCP server** provided in the mcp_agent folder. Go to the SageMaker terminal from this notebook and navigate to the agent-mcp foler and run the below command:

python creditcheck_server_http.py

This will start the MCP server that can accessible through this url: "http://0.0.0.0:8080/mcp".

We will connect to this server through Strands MCP client, list the tools provided by the MCP server and add the tools to the tool list provided the agent as shown int he cell below

In [51]:
# Create MCP HTTP client
# Replace with your actual MCP server URL
mcp_client = MCPClient(lambda: streamablehttp_client(
    url="http://0.0.0.0:8080/mcp"  # Your MCP server URL
    #streaming=True  # Enable streaming
))


Create the supervisor agent and provide both custom tools as well as the MCP tools

In [52]:

def create_supervisor_agent():
    """
    Create a supervisor agent that coordinates between the specialized agents
    and integrates MCP tools
    
    Returns:
        Agent: The supervisor agent
    """
    # Connect to MCP server and get tools
    with mcp_client:
        try:
            # Get MCP tools
            mcp_tools = mcp_client.list_tools_sync()
            logger.info(f"Loaded {len(mcp_tools)} MCP tools")
            
            # Define supervisor system prompt
            supervisor_system_prompt = """
            Your role is to provide a unified experience for all things related to mortgages. You are a supervisor who oversees answering
            customer questions related to general mortgages questions and queries about the existing mortgage.

            For general questions, use the answer_general_mortgage_questions tool.
            For questions on existing mortgage, use the answer_existing_mortgage_questions tool.
            If asked for a complicated calculation, use your code interpreter to be sure it's done accurately.
            
            You also have access to MCP tools that can perform additional function to get the credit score of existing customer.
            Use these tools when appropriate for the customer's query.
            
            Synthesize the details from the response of the tools used into a comprehensive answer provided back to the customer.
            """
            
            # Combine custom tools with MCP tools
            all_tools = [
                answer_general_mortgage_questions, 
                answer_existing_mortgage_questions, 
                calculator
            ] + mcp_tools
            
            # Create the supervisor agent
            supervisor = Agent(
                model="anthropic.claude-3-sonnet-20240229-v1:0",
                system_prompt=supervisor_system_prompt,
                tools=all_tools
            )
            
            return supervisor
            
        except Exception as e:
            logger.error(f"Error creating supervisor agent with MCP tools: {str(e)}")
            
            # Fallback to creating supervisor without MCP tools
            logger.info("Creating supervisor agent without MCP tools")
            supervisor = Agent(
                model="anthropic.claude-3-sonnet-20240229-v1:0",
                system_prompt="""
                Your role is to provide a unified experience for all things related to mortgages. You are a supervisor who oversees answering
                customer questions related to general mortgages questions and queries about the existing mortgage.

                For general questions, use the answer_general_mortgage_questions tool.
                For questions on existing mortgage, use the answer_existing_mortgage_questions tool.
                If asked for a complicated calculation, use your code interpreter to be sure it's done accurately.
                
                Synthesize the details from the response of the tools used into a comprehensive answer provided back to the customer.
                """,
                tools=[answer_general_mortgage_questions, answer_existing_mortgage_questions, calculator]
            )
            
            return supervisor

In [53]:
# need to run this on terminal ->python creditcheck_server_http.py

In [54]:
def answer_customer_query(query):
    """
    Process the customer query through the multi-agent system with MCP integration
    
    Args:
        query: The customer's query
        
    Returns:
        str: The response from the supervisor agent
    """
    # Create supervisor agent with MCP tools
    supervisor = create_supervisor_agent()
    
    # Process the query
    return supervisor(f"Provide a comprehensive answer for this query: {query}")

In [55]:
#test the supervisor agent
print("\n\nInvoking supervisor agent...\n\n")

requests = [#"I am customer: 3345, when’s my next payment due?",
            "what is my credit score, my customer id is 1111"
            #"what’s my balance after that payment, and what rate am I paying?",
            #"why do so many people choose a 30-year mortgage??",
            #"did you receive my employment verification doc yet? i sent it last week",
            #"i’m getting ready to lock in on a rate. what have the rates looked like in last couple weeks?",
            # "great. if i use the highest of those rates for $500K for 15 years, what’s my payment?"
            ]

for request in requests:
    print(f"\n\nRequest: {request}\n\n")
    result = answer_customer_query(request)
    time.sleep(10)
    print(result)

[2025-06-18 11:46:23,850] p2743 {streamable_http.py:465} INFO - Connecting to StreamableHTTP endpoint: http://0.0.0.0:8080/mcp
[2025-06-18 11:46:23,867] p2743 {_client.py:1786} INFO - HTTP Request: POST http://0.0.0.0:8080/mcp "HTTP/1.1 307 Temporary Redirect"
[2025-06-18 11:46:23,876] p2743 {_client.py:1786} INFO - HTTP Request: POST http://0.0.0.0:8080/mcp/ "HTTP/1.1 200 OK"
[2025-06-18 11:46:23,876] p2743 {streamable_http.py:140} INFO - Received session ID: d20cecb3d97546b8b8e49c1188c5d606
[2025-06-18 11:46:23,886] p2743 {_client.py:1786} INFO - HTTP Request: POST http://0.0.0.0:8080/mcp "HTTP/1.1 307 Temporary Redirect"
[2025-06-18 11:46:23,888] p2743 {_client.py:1786} INFO - HTTP Request: GET http://0.0.0.0:8080/mcp "HTTP/1.1 307 Temporary Redirect"
[2025-06-18 11:46:23,892] p2743 {_client.py:1786} INFO - HTTP Request: POST http://0.0.0.0:8080/mcp/ "HTTP/1.1 202 Accepted"
[2025-06-18 11:46:23,896] p2743 {_client.py:1786} INFO - HTTP Request: GET http://0.0.0.0:8080/mcp/ "HTTP/1.1 



Invoking supervisor agent...




Request: what is my credit score, my customer id is 1111



Tool #1: credit_check


[2025-06-18 11:46:26,688] p2743 {tool_handler.py:108} ERROR - tool_name=<credit_check> | failed to process tool
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/strands/handlers/tool_handler.py", line 105, in process
    return tool_func.invoke(tool, **kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/strands/tools/mcp/mcp_agent_tool.py", line 83, in invoke
    return self.mcp_client.call_tool_sync(
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 189, in call_tool_sync
    raise MCPClientInitializationError(CLIENT_SESSION_NOT_RUNNING_ERROR_MESSAGE)
strands.types.exceptions.MCPClientInitializationError: the client session is not running. Ensure the agent is used within the MCP client context manager. For more information see: https://strandsagents.com/latest/user-guide/concepts/tools/mcp-tools/#mcpclientinitializationerror




I apologize, there seems to be an issue connecting to the credit check tool to retrieve your credit score based on your customer ID 1111. As an AI assistant without direct access to real financial systems, I cannot actually look up your credit score. 

However, I can provide some general information about credit scores:

- Credit scores range from 300-850 for FICO scores and 501-990 for VantageScore models. Higher scores indicate lower credit risk.

- Factors that influence credit scores include payment history, amounts owed, length of credit history, new credit applications, and credit mix.

- Checking your own credit reports and scores from the major bureaus (Experian, Equifax, TransUnion) does not negatively impact your credit.

- Most lenders use credit scores as part of their criteria for approving loans and setting interest rates. Higher scores generally qualify for better rates.

- You are entitled to receive free annual credit reports from each bureau at AnnualCreditReport.co